# Fine-grained Food Classification using Keras

This Colab uses [FoodX-251](https://www.kaggle.com/c/ifood-2019-fgvc6/data) dataset. This dataset has 251 fine-grained (prepared) food categories with 118475 training images, 11994 validation images and 28377 test images.

Food classification is a challenging problem due to the large number of food categories, high visual similarity between different food categories. 

It’s multi-class classification problem to predict the 251 fine-grained food-category label given a food image.

The below three lines ensure that any edits to libraries you make are reloaded here automatically, and also that any charts or images displayed are shown in this notebook.

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Import necessary libraries

In [0]:
# System related libraries
import os
import datetime
# Handle table-like data and matrices
import numpy as np
import pandas as pd
# keras libraries
import keras
import tensorflow as tf
from keras import optimizers
from keras.layers import *
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, Callback, LearningRateScheduler, CSVLogger
# sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Visualization libraries
import matplotlib.pyplot as plt
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/
%matplotlib inline

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Add the `train_set, val_set, test_set, train_labels, val_labels` files to the drive instance

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
! ls '/content/gdrive/Shared drives/CMPE 258: Deep Learning/dataset/iFood'

test_set      train_labels.csv	train_set1     val_labels.csv  val_set.zip
test_set.zip  train_set		train_set.zip  val_set


Unzip the `train_set.zip`

In [0]:
!unzip '/content/gdrive/Shared drives/CMPE 258: Deep Learning/dataset/iFood/train_set.zip'

Streaming output truncated to the last 5000 lines.
  inflating: train_set/train_034206.jpg  
  inflating: train_set/train_054163.jpg  
  inflating: train_set/train_009636.jpg  
  inflating: train_set/train_013213.jpg  
  inflating: train_set/train_077657.jpg  
  inflating: train_set/train_026194.jpg  
  inflating: train_set/train_000231.jpg  
  inflating: train_set/train_060476.jpg  
  inflating: train_set/train_093616.jpg  
  inflating: train_set/train_054918.jpg  
  inflating: train_set/train_025201.jpg  
  inflating: train_set/train_022071.jpg  
  inflating: train_set/train_071302.jpg  
  inflating: train_set/train_035078.jpg  
  inflating: train_set/train_019982.jpg  
  inflating: train_set/train_060417.jpg  
  inflating: train_set/train_081457.jpg  
  inflating: train_set/train_021311.jpg  
  inflating: train_set/train_077393.jpg  
  inflating: train_set/train_102642.jpg  
  inflating: train_set/train_062665.jpg  
  inflating: train_set/train_111096.jpg  
  inflating: train_set/tr

Unzip the `test_set.zip`

In [0]:
!unzip '/content/gdrive/Shared drives/CMPE 258: Deep Learning/dataset/iFood/test_set.zip'

Streaming output truncated to the last 5000 lines.
  inflating: test_set/test_022242.jpg  
  inflating: test_set/test_021981.jpg  
  inflating: test_set/test_009481.jpg  
  inflating: test_set/test_013960.jpg  
  inflating: test_set/test_002362.jpg  
  inflating: test_set/test_005803.jpg  
  inflating: test_set/test_015388.jpg  
  inflating: test_set/test_002764.jpg  
  inflating: test_set/test_021964.jpg  
  inflating: test_set/test_013738.jpg  
  inflating: test_set/test_016448.jpg  
  inflating: test_set/test_022310.jpg  
  inflating: test_set/test_002203.jpg  
  inflating: test_set/test_003963.jpg  
  inflating: test_set/test_022431.jpg  
  inflating: test_set/test_003303.jpg  
  inflating: test_set/test_006227.jpg  
  inflating: test_set/test_016860.jpg  
  inflating: test_set/test_009457.jpg  
  inflating: test_set/test_020554.jpg  
  inflating: test_set/test_003655.jpg  
  inflating: test_set/test_006939.jpg  
  inflating: test_set/test_003241.jpg  
  inflating: test_set/test_02

Unzip the `val_set.zip`

In [0]:
!unzip '/content/gdrive/Shared drives/CMPE 258: Deep Learning/dataset/iFood/val_set.zip'

Streaming output truncated to the last 5000 lines.
  inflating: val_set/val_004199.jpg  
  inflating: val_set/val_000848.jpg  
  inflating: val_set/val_007717.jpg  
  inflating: val_set/val_007130.jpg  
  inflating: val_set/val_002708.jpg  
  inflating: val_set/val_001085.jpg  
  inflating: val_set/val_005015.jpg  
  inflating: val_set/val_012038.jpg  
  inflating: val_set/val_010613.jpg  
  inflating: val_set/val_010968.jpg  
  inflating: val_set/val_007924.jpg  
  inflating: val_set/val_010303.jpg  
  inflating: val_set/val_001124.jpg  
  inflating: val_set/val_011892.jpg  
  inflating: val_set/val_006131.jpg  
  inflating: val_set/val_012008.jpg  
  inflating: val_set/val_004498.jpg  
  inflating: val_set/val_008342.jpg  
  inflating: val_set/val_005417.jpg  
  inflating: val_set/val_012107.jpg  
  inflating: val_set/val_002339.jpg  
  inflating: val_set/val_010129.jpg  
  inflating: val_set/val_000308.jpg  
  inflating: val_set/val_004271.jpg  
  inflating: val_set/val_000598.jpg  

In [0]:
# Read data file into colaboratory
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1zILNhiLvCZnP7GVb7Sl4DtcVxI1DiYFh'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train_labels.csv')

In [0]:
link = 'https://drive.google.com/open?id=1PgXgtUL3TurOyZpL9f2xafbEGQo3y8e9'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('val_labels.csv')

In [0]:
import os
print(os.listdir("./"))

['.config', 'gdrive', 'train_labels.csv', 'adc.json', 'val_labels.csv', 'test_set', 'val_set', 'train_set', 'sample_data']


In [0]:
df_train = pd.read_csv('./train_labels.csv')
df_val   = pd.read_csv('./val_labels.csv')

In [0]:
# convert class to string
df_train['label'] = df_train['label'].astype(str)
df_val['label'] = df_val['label'].astype(str)

df_train.head()

,img_name,label
0,train_101733.jpg,211
1,train_101734.jpg,211
2,train_101735.jpg,211
3,train_101736.jpg,211
4,train_101737.jpg,211


In [0]:
df_train.shape, df_val.shape

((118475, 2), (11994, 2))

In [0]:
# network parameters
img_width, img_height = 299, 299
batch_size  = 16
epochs      = 150
target_classes = 251


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
val_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_dataframe(
                df_train,
                x_col='img_name',
                y_col='label',
                directory='./train_set/',  # this is the target directory
                shuffle = True,
                class_mode='categorical',
                target_size=(img_height, img_width),  # all images will be resized to 299 x 299
                batch_size=batch_size)  # since we use binary_crossentropy loss, we need binary labels

Found 118475 validated image filenames belonging to 251 classes.


In [0]:
val_generator = val_datagen.flow_from_dataframe(
                df_val,
                x_col='img_name',
                y_col='label',
                directory='./val_set/',  # this is the target directory
                shuffle = False,
                class_mode='categorical',
                target_size=(img_height, img_width),  # all images will be resized to 150x150
                batch_size=batch_size)  # since we use binary_crossentropy loss, we need binary labels

Found 11994 validated image filenames belonging to 251 classes.


In [0]:
inception = InceptionV3(weights='imagenet', include_top=False)
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(251,kernel_regularizer=l2(0.005), activation='softmax')(x)

model = Model(inputs=inception.input, outputs=predictions)

87916544/87910968 [==============================] - 4s 0us/step


Finetune Inceptionv3 pretrained model with L2 kernel regularizer with a penalty of 0.05 in FC layer, SGD optimizer with learning rate of 0.0001 and 0.9 momentum.

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

Compile the model

In [0]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

Trainig the model for 150 epochs.

In [0]:
checkpointer = ModelCheckpoint(filepath='best_model_251class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_251class.log')
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit_generator(train_generator,
                    steps_per_epoch = len(train_generator) // batch_size,
                    validation_data=val_generator,
                    validation_steps= len(val_generator) // batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer,tensorboard_callback])

model.save('model_trained_251class.hdf5')

Epoch 1/150
  2/462 [..............................] - ETA: 1:45:16 - loss: 7.3090 - accuracy: 0.0000e+00

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.362774). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


462/462 [==============================] - 254s 550ms/step - loss: 7.2577 - accuracy: 0.0047 - val_loss: 7.1673 - val_accuracy: 0.0082

Epoch 00001: val_loss improved from inf to 7.16733, saving model to best_model_251class.hdf5
Epoch 2/150
462/462 [==============================] - 216s 467ms/step - loss: 7.1946 - accuracy: 0.0062 - val_loss: 7.0648 - val_accuracy: 0.0068

Epoch 00002: val_loss improved from 7.16733 to 7.06476, saving model to best_model_251class.hdf5
Epoch 3/150
462/462 [==============================] - 216s 468ms/step - loss: 7.1436 - accuracy: 0.0080 - val_loss: 7.0872 - val_accuracy: 0.0082

Epoch 00003: val_loss did not improve from 7.06476
Epoch 4/150
462/462 [==============================] - 215s 465ms/step - loss: 7.0980 - accuracy: 0.0124 - val_loss: 7.0172 - val_accuracy: 0.0109

Epoch 00004: val_loss improved from 7.06476 to 7.01719, saving model to best_model_251class.hdf5
Epoch 5/150
462/462 [==============================] - 214s 462ms/step - loss: 7.0

In [0]:
new_model = keras.models.load_model('best_model_251class.hdf5')

Training the model for more 50 epochs.

In [0]:
history2 = new_model.fit_generator(train_generator,
                    steps_per_epoch = len(train_generator) // batch_size,
                    validation_data=val_generator,
                    validation_steps= len(val_generator) // batch_size,
                    epochs=50,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer,tensorboard_callback])

model.save('model_trained_251class.hdf5')

Epoch 1/50
  2/462 [..............................] - ETA: 1:12:31 - loss: 2.5849 - accuracy: 0.6250

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (1.686214). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


462/462 [==============================] - 231s 500ms/step - loss: 2.8257 - accuracy: 0.4946 - val_loss: 2.3235 - val_accuracy: 0.5516

Epoch 00001: val_loss did not improve from 1.57663
Epoch 2/50
462/462 [==============================] - 207s 448ms/step - loss: 2.7979 - accuracy: 0.4976 - val_loss: 2.1769 - val_accuracy: 0.5285

Epoch 00002: val_loss did not improve from 1.57663
Epoch 3/50
462/462 [==============================] - 206s 447ms/step - loss: 2.8180 - accuracy: 0.4959 - val_loss: 2.8281 - val_accuracy: 0.5367

Epoch 00003: val_loss did not improve from 1.57663
Epoch 4/50
462/462 [==============================] - 206s 446ms/step - loss: 2.8103 - accuracy: 0.4978 - val_loss: 2.0551 - val_accuracy: 0.6019

Epoch 00004: val_loss did not improve from 1.57663
Epoch 5/50
462/462 [==============================] - 205s 444ms/step - loss: 2.8292 - accuracy: 0.4881 - val_loss: 2.2872 - val_accuracy: 0.6005

Epoch 00005: val_loss did not improve from 1.57663
Epoch 6/50
462/462 [=